In [1]:
import os
import sys
import torch
import faiss
import numpy as np
import spacy
import gc
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset, Dataset
# Registers the factory
from scispacy.linking import EntityLinker 

/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [2]:
MODEL_NAME = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
BATCH_SIZE = 1024       # GPU Batch size
INDEX_CHUNK_SIZE = 50000 # RAM Batch size
TOP_K = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
FAISS_INDEX_STRING = "IVF1024,PQ64" 

print(f"Running on: {DEVICE}")

Running on: cuda


In [3]:

nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
linker = nlp.get_pipe("scispacy_linker")
kb = linker.kb
print(f"   ✅ UMLS KB Loaded. Total concepts: {len(kb.cui_to_entity)}")

print("2. Loading SapBERT Model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()
print("   ✅ SapBERT Loaded.")

   

/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.htm

   ✅ UMLS KB Loaded. Total concepts: 3920422
2. Loading SapBERT Model...
   ✅ SapBERT Loaded.


In [9]:
kb.cui_to_entity['C0000005']

CUI: C0000005, Name: (131)I-Macroaggregated Albumin
Definition: None
TUI(s): T116, T121, T130
Aliases: (total: 1): 
	 (131)I-MAA

In [1]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_lg")

# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

doc = nlp("Smoking cures cancer")

# Let's look at a random entity!
entity = doc.ents[1]

print("Name: ", entity)


# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])



Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from

Name:  cancer
CUI: C0006826, Name: Malignant Neoplasms
Definition: A tumor composed of atypical neoplastic, often pleomorphic cells that invade other tissues. Malignant neoplasms often metastasize to distant anatomic sites and may recur after excision. The most common malignant neoplasms are carcinomas, Hodgkin and non-Hodgkin lymphomas, leukemias, melanomas, and sarcomas.
TUI(s): T191
Aliases (abbreviated, total: 44): 
	 Neoplasms, Malignant, Cancers, Tumor, malignant, NOS, Malignant neoplasm without specification of site, Neoplastic disease, malignant, CA - Cancer, Malignancy, unspecified site, neoplasm/cancer, malignant neoplasm, Tumor, malignant
CUI: C0998265, Name: Cancer Genus
Definition: None
TUI(s): T204
Aliases: (total: 2): 
	 Cancer, crab
CUI: C1306459, Name: Primary malignant neoplasm
Definition: A malignant tumor at the original site of growth.
TUI(s): T191
Aliases (abbreviated, total: 19): 
	 Malignant neoplasm, primary (morphologic abnormality), malignant neoplasm, Tumor,

: 